### Обучение и сохранение модели

Датасет: https://www.kaggle.com/fedesoriano/stroke-prediction-dataset

In [1]:
import numpy as np
import pandas as pd
import dill

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline, FeatureUnion

In [2]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')
df.head(3)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1


In [3]:
df.tail(3)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0
5109,44679,Female,44.0,0,0,Yes,Govt_job,Urban,85.28,26.2,Unknown,0


In [4]:
df = df.dropna()

In [5]:
features = ['age', 'hypertension', 'heart_disease', 'smoking_status']
target = 'stroke'

In [6]:
df['smoking_status'] = df['smoking_status'].replace({'never smoked': 1, 'Unknown': 1, 'formerly smoked': 0, 'smokes': 0}).astype('float32')

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df, 
                                                    df['stroke'], test_size=0.3, random_state=1)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [8]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [9]:
continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in features:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
feats = FeatureUnion(continuos_transformers + cat_transformers + base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[ 1.02300085, -0.32266563, -0.22744102, -1.45629644],
       [-1.68882743, -0.32266563, -0.22744102,  0.6866734 ],
       [-0.6663348 , -0.32266563, -0.22744102,  0.6866734 ],
       ...,
       [-0.44405379, -0.32266563, -0.22744102,  0.6866734 ],
       [-1.28872162, -0.32266563, -0.22744102,  0.6866734 ],
       [ 1.06745705, -0.32266563, -0.22744102, -1.45629644]])

In [10]:
pipeline = Pipeline([
    ('features', feats),
    ('classifier', RandomForestClassifier(random_state=1)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('age',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='age')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('hypertension',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='hypertension')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('heart_disease',
                                                 Pipeline(steps=[('selector',
                     

In [11]:
with open("randomforest_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)